In [6]:
import numpy as np
import pandas as pd
import random
from datetime import datetime
import MetaTrader5 as mt5
import time
import concurrent.futures
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from imblearn.under_sampling import RandomUnderSampler
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [7]:
BACKWARD = datetime(2024, 12, 18)
EXAMWARD = datetime(2025, 2, 18)

def getData(symbol, timeframe, start, end,columns):
    attempt = 0
    retries_limit = 100
    while attempt < retries_limit:
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            mt5.shutdown()
            return None
        instrument_count = mt5.symbols_total()
        print(mt5.terminal_info())
        print(mt5.terminal_info()._asdict())
        if instrument_count > 0:
            print(f"Number of instruments in the terminal: {instrument_count}")
        else:
            print("No instruments in the terminal")

        rates = mt5.copy_rates_range(symbol, timeframe, start, end)
        if(rates is None):
            print(mt5.last_error())
        mt5.shutdown()
        print(len(rates))
        if rates is None or len(rates) == 0:
            print(f"Data for {symbol} not available (attempt {attempt+1})")
            attempt += 1
            time.sleep(1)
        else:
            raw_data = pd.DataFrame(rates[:-1], columns=columns)
            break
    if raw_data is None:
        print(f"Data retrieval failed after {retries_limit} attempts")
        return None
    return raw_data

In [8]:
def data_preprocessing(raw_data,count):
    dataset = raw_data["close"]-raw_data["open"]
    input_data = np.empty((0,2*count+1))
    for i in range(0, len(dataset)-count-1):
        temp_data = dataset[i:i+count]
        fft_signal = np.fft.fft(temp_data)
        magnitude = np.abs(fft_signal)
        phase = np.angle(fft_signal)
        if dataset[i+count+1] > 0:
            roi = 1
        else:
            roi = 0
        result = np.hstack((magnitude, phase))
        result = np.append(result, roi)
        input_data =np.vstack((input_data,result))
    return input_data
        
    
    

In [10]:

raw_data = getData('XAUUSD', mt5.TIMEFRAME_M30, BACKWARD, EXAMWARD,["time","open","close"])
raw_data["time"] = pd.to_datetime(raw_data["time"], unit='s')


TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=4872, maxbars=100000, codepage=936, ping_last=185143, community_balance=0.0, retransmission=5.704118607974359, company='MetaQuotes Ltd.', name='MetaTrader 5', language='Chinese (Simplified)', path='D:\\MQL5', data_path='D:\\MQL5', commondata_path='C:\\Users\\Reolink\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
{'community_account': False, 'community_connection': False, 'connected': True, 'dlls_allowed': False, 'trade_allowed': False, 'tradeapi_disabled': False, 'email_enabled': False, 'ftp_enabled': False, 'notifications_enabled': False, 'mqid': False, 'build': 4872, 'maxbars': 100000, 'codepage': 936, 'ping_last': 185143, 'community_balance': 0.0, 'retransmission': 5.704118607974359, 'company': 'MetaQuotes Ltd.', 'name': 'MetaTrader 5', 'language': 

In [30]:
input_data = data_preprocessing(raw_data,10)
print(input_data.shape)

(57311, 21)


In [31]:
def xgb_model(input_data,rounds):
    x_train, x_test, y_train, y_test = train_test_split(input_data[:,:-1], input_data[:,-1], test_size=0.2)
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dtest = xgb.DMatrix(x_test, label=y_test)
    params= {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'eta': 0.05,
        'max_depth': 6,
    }
    model = xgb.train(params, dtrain, rounds)
    y_pred = model.predict(dtest)
    y_pred_binary = [1 if i > 0.9 else 0 for i in y_pred]
    accuracy = accuracy_score(y_test, y_pred_binary)
    print("accuracy：", accuracy)
    return y_pred_binary
    


In [32]:
y_pred_binary = xgb_model(input_data,100)
print(len(y_pred_binary))

accuracy： 0.501090464974265
11463


In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('D:/MQL5/MQL5/data.sqlite')


In [ ]:
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS XAUUSD (time INTEGER , prediction INTEGER)")

In [16]:
cursor.execute("INSERT INTO XAUUSD (time, prediction) VALUES (?, ?)", (1, 10))
cursor.execute("INSERT INTO XAUUSD (time, prediction) VALUES (?, ?)", (2, 20))
cursor.execute("INSERT INTO XAUUSD (time, prediction) VALUES (?, ?)", (3, 30))
conn.commit()
